In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism*2)
dbutils.library.installPyPI('ordered_set')
dbutils.library.installPyPI('fredapi')
dbutils.library.installPyPI('azure-storage-file-datalake' ,version="12.2.0" )

dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook-scoped python libraries.
dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook-scoped python libraries.
dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook-scoped python libraries.
Out[1]: True

In [0]:
import pandas as pd
from datetime import datetime
from ordered_set import OrderedSet
from fredapi import Fred as fred
from pyspark.sql.functions import col , row_number , year , to_timestamp, regexp_replace , lit , to_date
from pyspark.sql.types import * 
from pyspark.sql import Window
from delta.tables import *
from pathlib import Path
import os 

start_date = datetime(1981, 1, 1)
end_date = datetime.today().date()

fred_api_key = dbutils.secrets.get(scope ="am-da-kv-general-user" ,key="na-da-s-FredAPIKey")
fred = fred(api_key=fred_api_key) 

In [0]:
##Price Indices
cpiausl = "CPIAUCSL" #Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
cpilfesl = "CPILFESL" #Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average
dcoilwtico = "DCOILWTICO" #Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma
ppiaco = "PPIACO" #Producer Price Index by Commodity: All Commodities
wpu10 = "WPU10" #Producer Price Index by Commodity: Metals and Metal Products
wps057303 = "WPS057303" #Producer Price Index by Commodity: Fuels and Related Products and Power: No. 2 Diesel Fuel
wps0571 = "WPS0571" #Producer Price Index by Commodity: Fuels and Related Products and Power: Gasoline

##Personal Income
mehoinusa672n = "MEHOINUSA672N" #Real Median Household Income in the United States
pcedg = "PCEDG" #Personal Consumption Expenditures: Durable Goods
pce = 'PCE' ##'Personal Consumption Expenditure'
psavert = 'PSAVERT'  ##'Personal Saving Rate'
dspi = "DSPI" #Disposable Personal Income
dspic96 = 'DSPIC96'  ##'Real Disposable Personal Income'

##Unemployment
unrate = 'UNRATE' ##'Unemployment Rate'
nrou = "NROU" #Noncyclical Rate of Unemployment
civpart = "CIVPART" #Labor Force Participation Rate
emratio = "EMRATIO" #Employment-Population Ratio
unemploy = "UNEMPLOY" #Unemployment Level
payems = "PAYEMS" #All Employees, Total Nonfarm
manemp = "MANEMP" #All Employees, Manufacturing
icsa = "ICSA" #Initial Claims
ic4wsa = "IC4WSA" #4-Week Moving Average of Initial Claims

##Monetary
m2sl = 'M2SL'  ##'M2'
rbusbis = 'RBUSBIS' ##'Real Broad Effective Exchange Rate for United States'
gs10 = 'GS10' ##'Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis'
fedfunds = 'FEDFUNDS' ##'Federal Funds Effective Rate'
ttlcons = 'TTLCONS' ##'Total Construction Spending: Total Construction in the United States'
indpro='INDPRO' ##''Industrial Production: Total Index'
gfdebtn = "GFDEBTN" #Federal Debt: Total Public Debt
dff = "DFF" #Federal Funds Effective Rate
dtb = "DTB3"  #3-Month Treasury Bill Secondary Market Rate, Discount Basis
dgs5 = "DGS5" #Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity, Quoted on an Investment Basis
dgs10 = "DGS10" #Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis
dgs30 = "DGS30" #Market Yield on U.S. Treasury Securities at 30-Year Constant Maturity, Quoted on an Investment Basis
t5yie = "T5YIE" #5-Year Breakeven Inflation Rate
t10yie = "T10YIE" #10-Year Breakeven Inflation Rate
t5yifr = "T5YIFR" #5-Year, 5-Year Forward Inflation Expectation Rate
dprime = "DPRIME" #Bank Prime Loan Rate

##Other indicators
tcu = "TCU" #Capacity Utilization: Total Index
rrsfs = "RRSFS" #Advance Real Retail and Food Services Sales
houst = "HOUST" #New Privately-Owned Housing Units Started: Total Units
cp = "CP" #Corporate Profits After Tax (without IVA and CCAdj)
totci = "TOTCI" #Commercial and Industrial Loans, All Commercial Banks
ttlcons = 'TTLCONS' ##'Total Construction Spending: Total Construction in the United States'
indpro = 'INDPRO' ##''Industrial Production: Total Index'

In [0]:
#use variables defined above in previous cell, list inside of [ ... ] in the order you want the columns
series_names = [##Price Indices
cpiausl, #Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
cpilfesl, #Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average
dcoilwtico,#Crude Oil Prices: West Texas Intermediate (WTI) - Cushing, Oklahoma
ppiaco, #Producer Price Index by Commodity: All Commodities
wpu10, #Producer Price Index by Commodity: Metals and Metal Products
wps057303, #Producer Price Index by Commodity: Fuels and Related Products and Power: No. 2 Diesel Fuel
wps0571, #Producer Price Index by Commodity: Fuels and Related Products and Power: Gasoline
##Personal Income
mehoinusa672n, #Real Median Household Income in the United States
pcedg, #Personal Consumption Expenditures: Durable Goods
pce, ##'Personal Consumption Expenditure'
psavert,  ##'Personal Saving Rate'
dspi, #Disposable Personal Income
dspic96,  ##'Real Disposable Personal Income'

##Unemployment
unrate, ##'Unemployment Rate'
nrou, #Noncyclical Rate of Unemployment
civpart, #Labor Force Participation Rate
emratio, #Employment-Population Ratio
unemploy, #Unemployment Level
payems, #All Employees, Total Nonfarm
manemp,#All Employees, Manufacturing
icsa, #Initial Claims
ic4wsa, #4-Week Moving Average of Initial Claims

##Monetary
m2sl,  ##'M2'
rbusbis, ##'Real Broad Effective Exchange Rate for United States'
gs10, ##'Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis'
fedfunds, ##'Federal Funds Effective Rate'
ttlcons, ##'Total Construction Spending: Total Construction in the United States'
indpro, ##''Industrial Production: Total Index'
gfdebtn, #Federal Debt: Total Public Debt
dff, #Federal Funds Effective Rate
dtb,  #3-Month Treasury Bill Secondary Market Rate, Discount Basis
dgs5, #Market Yield on U.S. Treasury Securities at 5-Year Constant Maturity, Quoted on an Investment Basis
dgs10, #Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis
dgs30,#Market Yield on U.S. Treasury Securities at 30-Year Constant Maturity, Quoted on an Investment Basis
t5yie, #5-Year Breakeven Inflation Rate
t10yie, #10-Year Breakeven Inflation Rate
t5yifr, #5-Year, 5-Year Forward Inflation Expectation Rate
dprime, #Bank Prime Loan Rate

##Other indicators
tcu, #Capacity Utilization: Total Index
rrsfs, #Advance Real Retail and Food Services Sales
houst, #New Privately-Owned Housing Units Started: Total Units
cp, #Corporate Profits After Tax (without IVA and CCAdj)
totci, #Commercial and Industrial Loans, All Commercial Banks
ttlcons, ##'Total Construction Spending: Total Construction in the United States'
indpro ##''Industrial Production: Total Index'
]
print (series_names)

['CPIAUCSL', 'CPILFESL', 'DCOILWTICO', 'PPIACO', 'WPU10', 'WPS057303', 'WPS0571', 'MEHOINUSA672N', 'PCEDG', 'PCE', 'PSAVERT', 'DSPI', 'DSPIC96', 'UNRATE', 'NROU', 'CIVPART', 'EMRATIO', 'UNEMPLOY', 'PAYEMS', 'MANEMP', 'ICSA', 'IC4WSA', 'M2SL', 'RBUSBIS', 'GS10', 'FEDFUNDS', 'TTLCONS', 'INDPRO', 'GFDEBTN', 'DFF', 'DTB3', 'DGS5', 'DGS10', 'DGS30', 'T5YIE', 'T10YIE', 'T5YIFR', 'DPRIME', 'TCU', 'RRSFS', 'HOUST', 'CP', 'TOTCI', 'TTLCONS', 'INDPRO']

In [0]:
df = {}
for series in series_names:
    df[series] = fred.get_series_info(series)
datasets = pd.DataFrame(df).T.reset_index()
datasets["last_updated"] = str(datasets["last_updated"].str[:-3][0])
datasets["last_updated"] = pd.to_datetime(datasets["last_updated"])
datasets["observation_end"] = pd.to_datetime(datasets["observation_end"])
datasets["observation_start"] = pd.to_datetime(datasets["observation_start"])
datasets["realtime_end"] = pd.to_datetime(datasets["realtime_end"])
datasets["realtime_start"] = pd.to_datetime(datasets["realtime_start"])
datasets.head()

Out[5]:

,index,frequency,frequency_short,id,last_updated,notes,observation_end,observation_start,popularity,realtime_end,realtime_start,seasonal_adjustment,seasonal_adjustment_short,title,units,units_short
0,CPIAUCSL,Monthly,M,CPIAUCSL,2022-07-13 07:37:02,The Consumer Price Index for All Urban Consume...,2022-06-01,1947-01-01,93,2022-08-03,2022-08-03,Seasonally Adjusted,SA,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,Index 1982-1984=100
1,CPILFESL,Monthly,M,CPILFESL,2022-07-13 07:37:02,"The ""Consumer Price Index for All Urban Consum...",2022-06-01,1957-01-01,81,2022-08-03,2022-08-03,Seasonally Adjusted,SA,Consumer Price Index for All Urban Consumers: ...,Index 1982-1984=100,Index 1982-1984=100
2,DCOILWTICO,Daily,D,DCOILWTICO,2022-07-13 07:37:02,"Definitions, Sources and Explanatory Notes (ht...",2022-08-01,1986-01-02,81,2022-08-03,2022-08-03,Not Seasonally Adjusted,NSA,Crude Oil Prices: West Texas Intermediate (WTI...,Dollars per Barrel,$ per Barrel
3,PPIACO,Monthly,M,PPIACO,2022-07-13 07:37:02,NaN,2022-06-01,1913-01-01,79,2022-08-03,2022-08-03,Not Seasonally Adjusted,NSA,Producer Price Index by Commodity: All Commodi...,Index 1982=100,Index 1982=100
4,WPU10,Monthly,M,WPU10,2022-07-13 07:37:02,NaN,2022-06-01,1926-01-01,57,2022-08-03,2022-08-03,Not Seasonally Adjusted,NSA,Producer Price Index by Commodity: Metals and ...,Index 1982=100,Index 1982=100


In [0]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
schema = StructType([
  StructField('index',StringType(), False),
  StructField('id',StringType(), False),
  StructField('frequency',StringType(), True),
  StructField('frequency_short',StringType(), True),
  StructField('last_updated', DateType(), False),
  StructField('notes',StringType(), True),
  StructField('observation_end',DateType(), True),
  StructField('observation_start',DateType(), True),
  StructField('popularity',StringType(), True),
  StructField('realtime_end',DateType(), True),
  StructField('realtime_start',DateType(), True),
  StructField('seasonal_adjustment',StringType(), True),
  StructField('seasonal_adjustment_short',StringType(), True),
  StructField('title',StringType(), True),
  StructField('units',StringType(), True),
  StructField('units_short',StringType(), True)
])
datasets=spark.createDataFrame(data = datasets, schema = schema)
datasets.printSchema()

root
-- index: string (nullable = false)
-- id: string (nullable = false)
-- frequency: string (nullable = true)
-- frequency_short: string (nullable = true)
-- last_updated: date (nullable = false)
-- notes: string (nullable = true)
-- observation_end: date (nullable = true)
-- observation_start: date (nullable = true)
-- popularity: string (nullable = true)
-- realtime_end: date (nullable = true)
-- realtime_start: date (nullable = true)
-- seasonal_adjustment: string (nullable = true)
-- seasonal_adjustment_short: string (nullable = true)
-- title: string (nullable = true)
-- units: string (nullable = true)
-- units_short: string (nullable = true)

In [0]:
datasets.write\
              .partitionBy("id")\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/datalake_raw/public/fred/datasets")

spark.sql("CREATE TABLE if not exists ods.public_fred_datasets USING DELTA LOCATION '/mnt/datalake_raw/public/fred/datasets'")

Out[7]: DataFrame[]

In [0]:
%sql
SELECT * FROM ods.public_fred_datasets LIMIT 5

index,id,frequency,frequency_short,last_updated,notes,observation_end,observation_start,popularity,realtime_end,realtime_start,seasonal_adjustment,seasonal_adjustment_short,title,units,units_short
CPIAUCSL,Monthly,M,CPIAUCSL,2022-07-13,"The Consumer Price Index for All Urban Consumers: All Items (CPIAUCSL) is a price index of a basket of goods and services paid by urban consumers. Percent changes in the price index measure the inflation rate between any two time periods. The most common inflation metric is the percent change from one year ago. It can also represent the buying habits of urban consumers. This particular index includes roughly 88 percent of the total population, accounting for wage earners, clerical workers, technical workers, self-employed, short-term workers, unemployed, retirees, and those not in the labor force. The CPIs are based on prices for food, clothing, shelter, and fuels; transportation fares; service fees (e.g., water and sewer service); and sales taxes. Prices are collected monthly from about 4,000 housing units and approximately 26,000 retail establishments across 87 urban areas. To calculate the index, price changes are averaged with weights representing their importance in the spending of the particular group. The index measures price changes (as a percent change) from a predetermined reference date. In addition to the original unadjusted index distributed, the Bureau of Labor Statistics also releases a seasonally adjusted index. The unadjusted series reflects all factors that may influence a change in prices. However, it can be very useful to look at the seasonally adjusted CPI, which removes the effects of seasonal changes, such as weather, school year, production cycles, and holidays. The CPI can be used to recognize periods of inflation and deflation. Significant increases in the CPI within a short time frame might indicate a period of inflation, and significant decreases in CPI within a short time frame might indicate a period of deflation. However, because the CPI includes volatile food and oil prices, it might not be a reliable measure of inflationary and deflationary periods. For a more accurate detection, the core CPI (CPILFESL (https://fred.stlouisfed.org/series/CPILFESL)) is often used. When using the CPI, please note that it is not applicable to all consumers and should not be used to determine relative living costs. Additionally, the CPI is a statistical measure vulnerable to sampling error since it is based on a sample of prices and not the complete average. For more information on the consumer price indexes, see: Bureau of Economic Analysis. ""CPI Detailed Report."" (https://www.bls.gov/cpi/) 2013. Handbook of Methods (https://www.bls.gov/opub/hom/pdf/cpihom.pdf) Understanding the CPI: Frequently Asked Questions (https://www.bls.gov/cpi/questions-and-answers.htm)",2022-06-01,1947-01-01,93,2022-08-03,2022-08-03,Seasonally Adjusted,SA,Consumer Price Index for All Urban Consumers: All Items in U.S. City Average,Index 1982-1984=100,Index 1982-1984=100
CPILFESL,Monthly,M,CPILFESL,2022-07-13,"The ""Consumer Price Index for All Urban Consumers: All Items Less Food & Energy"" is an aggregate of prices paid by urban consumers for a typical basket of goods, excluding food and energy. This measurement, known as ""Core CPI,"" is widely used by economists because food and energy have very volatile prices. The Bureau of Labor Statistics defines and measures the official CPI, and more information can be found in the FAQ (https://www.bls.gov/cpi/questions-and-answers.htm) or in this article (https://www.bls.gov/opub/hom/pdf/cpihom.pdf).",2022-06-01,1957-01-01,81,2022-08-03,2022-08-03,Seasonally Adjusted,SA,Consumer Price Index for All Urban Consumers: All Items Less Food and Energy in U.S. City Average,Index 1982-1984=100,Index 1982-1984=100
PPIACO,Monthly,M,PPIACO,2022-07-13,null,2022-06-01,1913-01-01,79,2022-08-03,2022-08-03,Not Seasonally Adjusted,NSA,Producer Price Index by Commodity: All Comm

In [0]:
df = {}
#use variables defined above in previous cell
for series in series_names:
  df[series] = fred.get_series(series, start_date, end_date)
  df = pd.DataFrame(df)
#set numeric datatype from results
cols = df.columns
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
#move date index to fixed column and set datatype
df.reset_index(inplace=True)
df = df.rename(columns = {'index':'observation_date'})
df["observation_date"] = pd.to_datetime(df["observation_date"])
#display capture
df.head()

Out[8]: Index(['CPIAUCSL', 'CPILFESL', 'DCOILWTICO', 'PPIACO', 'WPU10', 'WPS057303',
 'WPS0571', 'MEHOINUSA672N', 'PCEDG', 'PCE', 'PSAVERT', 'DSPI',
 'DSPIC96', 'UNRATE', 'NROU', 'CIVPART', 'EMRATIO', 'UNEMPLOY', 'PAYEMS',
 'MANEMP', 'ICSA', 'IC4WSA', 'M2SL', 'RBUSBIS', 'GS10', 'FEDFUNDS',
 'TTLCONS', 'INDPRO', 'GFDEBTN', 'DFF', 'DTB3', 'DGS5', 'DGS10', 'DGS30',
 'T5YIE', 'T10YIE', 'T5YIFR', 'DPRIME', 'TCU', 'RRSFS', 'HOUST', 'CP',
 'TOTCI'],
 dtype='object')

In [0]:
from pyspark.sql import SparkSession
#Create PySpark SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .getOrCreate()
#Create PySpark DataFrame from Pandas
schema = StructType([
  StructField('observation_date', DateType(), False),
  StructField('CPIAUCSL',  FloatType(), True),
  StructField('CPILFESL',  FloatType(), True),
  StructField('DCOILWTICO',  FloatType(), True),
  StructField('PPIACO',  FloatType(), True),
  StructField('WPU10',  FloatType(), True),
  StructField('WPS057303',    FloatType(), True),    
  StructField('WPS0571',  FloatType(), True),
  StructField('MEHOINUSA672N',  FloatType(), True),
  StructField('PCEDG',  FloatType(), True),
  StructField('PCE',  FloatType(), True),
  StructField('PSAVERT',  FloatType(), True),
  StructField('DSPI', FloatType(), True),
  StructField('DSPIC96',  FloatType(), True),
  StructField('UNRATE',  FloatType(), True),
  StructField('NROU',  FloatType(), True),
  StructField('CIVPART',  FloatType(), True),
  StructField('EMRATIO',  FloatType(), True),
  StructField('UNEMPLOY', FloatType(), True), 
  StructField('PAYEMS', FloatType(), True),
  StructField('MANEMP',  FloatType(), True),
  StructField('ICSA',  FloatType(), True),
  StructField('IC4WSA', FloatType(), True), 
  StructField('M2SL',  FloatType(), True),
  StructField('RBUSBIS', FloatType(), True), 
  StructField('GS10',  FloatType(), True),
  StructField('FEDFUNDS', FloatType(), True),
  StructField('TTLCONS', FloatType(), True), 
  StructField('INDPRO',  FloatType(), True),
  StructField('GFDEBTN',  FloatType(), True),
  StructField('DFF',  FloatType(), True),
  StructField('DTB3', FloatType(), True), 
  StructField('DGS5',  FloatType(), True),
  StructField('DGS10', FloatType(), True) ,
  StructField('DGS30',  FloatType(), True),      
  StructField('T5YIE', FloatType(), True), 
  StructField('T10YIE',  FloatType(), True),
  StructField('T5YIFR',  FloatType(), True),
  StructField('DPRIME',  FloatType(), True),
  StructField('TCU',  FloatType(), True),
  StructField('RRSFS',  FloatType(), True),
  StructField('HOUST',  FloatType(), True),
  StructField('CP',  FloatType(), True) ,
  StructField('TOTCI', FloatType(), True)
])
datasets=spark.createDataFrame(data = df, schema = schema)
datasets.printSchema()

root
-- observation_date: date (nullable = false)
-- CPIAUCSL: float (nullable = true)
-- CPILFESL: float (nullable = true)
-- DCOILWTICO: float (nullable = true)
-- PPIACO: float (nullable = true)
-- WPU10: float (nullable = true)
-- WPS057303: float (nullable = true)
-- WPS0571: float (nullable = true)
-- MEHOINUSA672N: float (nullable = true)
-- PCEDG: float (nullable = true)
-- PCE: float (nullable = true)
-- PSAVERT: float (nullable = true)
-- DSPI: float (nullable = true)
-- DSPIC96: float (nullable = true)
-- UNRATE: float (nullable = true)
-- NROU: float (nullable = true)
-- CIVPART: float (nullable = true)
-- EMRATIO: float (nullable = true)
-- UNEMPLOY: float (nullable = true)
-- PAYEMS: float (nullable = true)
-- MANEMP: float (nullable = true)
-- ICSA: float (nullable = true)
-- IC4WSA: float (nullable = true)
-- M2SL: float (nullable = true)
-- RBUSBIS: float (nullable = true)
-- GS10: float (nullable = true)
-- FEDFUNDS: float (nullable = true)
-- TTLCONS: float (nullable = true)
-- INDPRO: float (nullable = true)
-- GFDEBTN: float (nullable = true)
-- DFF: float (nullable = true)
-- DTB3: float (nullable = true)
-- DGS5: float (nullable = true)
-- DGS10: float (nullable = true)
-- DGS30: float (nullable = true)
-- T5YIE: float (nullable = true)
-- T10YIE: float (nullable = true)
-- T5YIFR: float (nullable = true)
-- DPRIME: float (nullable = true)
-- TCU: float (nullable = true)
-- RRSFS: float (nullable = true)
-- HOUST: float (nullable = true)
-- CP: float (nullable = true)
-- TOTCI: float (nullable = true)

In [0]:
datasets.write\
              .partitionBy("observation_date")\
              .mode("overwrite")\
              .format("delta")\
              .option("overwriteSchema", "true")\
              .save("/mnt/datalake_raw/public/fred/data")

spark.sql("CREATE TABLE if not exists ods.public_fred_data USING DELTA LOCATION '/mnt/datalake_raw/public/fred/data'")

Out[16]: DataFrame[]

In [0]:
%sql
SELECT * FROM ods.public_fred_data ORDER BY 1 ASC LIMIT 5

observation_date,CPIAUCSL,CPILFESL,DCOILWTICO,PPIACO,WPU10,WPS057303,WPS0571,MEHOINUSA672N,PCEDG,PCE,PSAVERT,DSPI,DSPIC96,UNRATE,NROU,CIVPART,EMRATIO,UNEMPLOY,PAYEMS,MANEMP,ICSA,IC4WSA,M2SL,RBUSBIS,GS10,FEDFUNDS,TTLCONS,INDPRO,GFDEBTN,DFF,DTB3,DGS5,DGS10,DGS30,T5YIE,T10YIE,T5YIFR,DPRIME,TCU,RRSFS,HOUST,CP,TOTCI
1981-01-01,87.2,85.4,null,95.2,97.5,null,100.1,null,239.9,1870.0,10.9,2160.8,4980.4,7.5,6.187123,63.9,59.1,8071.0,91033.0,18639.0,null,null,1606.9,null,12.57,19.08,null,51.0656,964531.0,22.0,null,null,null,null,null,null,null,null,80.5752,null,1547.0,213.17,null
1981-02-01,88.0,85.9,null,96.1,97.5,null,104.1,null,247.3,1884.2,10.8,2175.9,4965.0,7.4,null,63.9,59.2,8051.0,91105.0,18613.0,null,null,1618.7,null,13.19,15.93,null,50.8554,null,17.25,null,null,null,null,null,null,null,null,80.0469,null,1246.0,null,null
1981-03-01,88.6,86.4,null,97.0,98.3,null,110.1,null,250.9,1902.9,10.8,2197.3,4979.0,7.4,null,64.1,59.4,7982.0,91210.0,18647.0,null,null,1636.6,null,13.12,14.7,null,51.1107,null,15.53,null,null,null,null,null,null,null,null,80.2479,null,1306.0,null,null
1981-04-01,89.1,87.0,null,98.0,99.1,null,114.9,null,241.7,1904.4,10.9,2200.7,4965.1,7.2,6.1783614,64.2,59.6,7869.0,91283.0,18711.0,null,null,1659.2,null,13.68,15.72,null,50.8864,971174.0,14.51,12.37,13.36,13.14,12.65,null,null,null,17.5,79.6948,null,1360.0,201.614,316.1307
1981-05-01,89.7,87.8,null,98.3,99.2,null,112.0,null,239.8,1913.8,11.0,2214.9,4974.8,7.5,null,64.3,59.5,8174.0,91296.0,18766.0,null,null,1664.2,null,14.1,18.52,null,51.1788,null,19.79,14.67,14.37,14.05,13.59,null,null,null,18.0,79.9489,null,1140.0,null,null


In [0]:
dbutils.notebook.exit("Job Completed Successfuly!")